In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as anim
import itertools as itr
import copy
from boundaries import WallBoundary, PressureBoundary, VelocityBoundary
from generators import WallGenerators
from simulation import Simulation
import random

# np.seterr(divide=None, invalid=None)  # For suppressing warnings. Totally optional.

yResolution = 24
xResolution = 36
initCondition = np.ones((yResolution, xResolution, Simulation.latticeSize)) / 9
walls = WallBoundary(yResolution, xResolution)
# walls.borderWall()
walls.cylindricalWall([12, 18], 5)
velocityInlet = [] # [VelocityBoundary(12, 2, 1, 1)]
pressureInlet = [PressureBoundary(12, 2, 1, 0, 1)]

simulation = Simulation(
    yResolution, xResolution, initCondition, walls, velocityBoundaries=velocityInlet, pressureBoundaries=pressureInlet
)

plt.imshow(simulation.density, cmap="hot", interpolation="nearest")

In [ ]:
yResolution = 24
xResolution = 36
initCondition = np.ones((yResolution, xResolution, Simulation.latticeSize)) / 9
walls = WallBoundary(yResolution, xResolution)
# walls.borderWall()

for i in random.sample(range(8), k = random.randint(1,8)):
    
    possiblePositions = [(int(yResolution/3), xResolution - 1), (0, int(xResolution/3)), (int(yResolution/3), 0), (yResolution - 1, int(xResolution/3)), (0, xResolution - 1), (0, 0), (yResolution - 1, 0), (yResolution - 1, xResolution - 1)]

    wallPos = possiblePositions[i]
    maxSize = int(min(yResolution, xResolution) * 0.4)
    minSize = int(min(yResolution, xResolution) * 0.2)
    if random.random() < 0.5:
        walls.cylindricalWall(wallPos, random.randint(minSize, maxSize))
    else:
        directions = [(1,-1),(1,1),(1,1),(-1,1),(1,-1),(1,1),(-1,1),(-1,-1)]
        endPos = (wallPos[0] + (random.randint(minSize, maxSize)*directions[i][0]), wallPos[1] + (random.randint(minSize, maxSize)*directions[i][1]))
        walls.filledStraightRectangularWall(wallPos, endPos)
    #print(f"{i} {wallPos} {endPos} {directions[i]}")

walls.borderWall()

velocityInlet = [VelocityBoundary(12, 2, 1, 1)]
#pressureInlet = [PressureBoundary(12, 2, 1, 0, 1)]

simulation = Simulation(
    yResolution, xResolution, initCondition, walls, velocityBoundaries=velocityInlet, pressureBoundaries=pressureInlet
)

plt.imshow(simulation.density, cmap="hot", interpolation="nearest")

In [ ]:
yResolution = 48
xResolution = 32
initCondition = np.ones((yResolution, xResolution, Simulation.latticeSize)) / 9
walls = WallBoundary(yResolution, xResolution)
# walls.borderWall()
walls.borderWall()
conditions = walls.generateRoom()
walls.generateIndex()
walls.generateACDirections()
walls.indexPossibleACPos()
print(conditions)
print(walls.possibleACIndex)
#velocityInlet = [VelocityBoundary(12, 2, 1, 1)]
#pressureInlet = [PressureBoundary(12, 2, 1, 0, 1)]

simulation = Simulation(
    yResolution, xResolution, initCondition, walls, velocityBoundaries=velocityInlet, pressureBoundaries=pressureInlet
)

plt.imshow(simulation.density, cmap="hot", interpolation="nearest")

In [ ]:
fig, ax = plt.subplots()
heatmap  = ax.imshow(walls.boundary, cmap="hot", interpolation="nearest")
fig.colorbar(heatmap)

In [ ]:
walls.generateIndex()
walls.generateACDirections()
walls.indexPossibleACPos()

In [ ]:
#walls.possibleACPos
fig, ax = plt.subplots()
heatmap  = ax.imshow(walls.invertedBoundary, cmap="hot", interpolation="nearest")
fig.colorbar(heatmap)
ax.scatter([i[1] for i in walls.possibleACIndex], [i[0] for i in walls.possibleACIndex])
print(walls.possibleACIndex)

# outPutDirectionsY = np.array([WallBoundary.unitVect[j][0] for i in walls.possibleACDirections for j in i])
# outPutDirectionsX = np.array([WallBoundary.unitVect[j][1] for i in walls.possibleACDirections for j in i])
# outPutPositionsX = [i[0] for index, i in enumerate(walls.possibleACPos) for j in range(len(walls.possibleACDirections[index]))]
# print(len(outPutPositionsX))
# outPutPositionsY = [i[1] for index, i in enumerate(walls.possibleACPos) for j in range(len(walls.possibleACDirections[index]))]
# quivermap = ax.quiver(outPutPositionsY, outPutPositionsX, 310*outPutDirectionsX, -310*outPutDirectionsY)

In [ ]:
walls.boundaryIndex
walls.boundary
from scipy.ndimage import convolve
def numerical_inverse(n):
    return int(n != 1)

applyall = np.vectorize(numerical_inverse)
inverted = applyall(walls.boundary.astype(int))
#print(inverted)
#plt.imshow(inverted, cmap="hot", interpolation="nearest")
padded_array = np.pad(inverted, pad_width=1, mode='constant', constant_values=0)
kernel = [[1,1,1],[1,0,1],[1,1,1]]
edgeMinesweeper = convolve(padded_array, kernel)[1:-1, 1:-1] 
edgesProcessed = np.logical_and(edgeMinesweeper >= 2, edgeMinesweeper <= 5)

edges = []
edgesX = []
edgesY = []
for i in walls.boundaryIndex:
    if (edgesProcessed[i[0], i[1]]):
        edges.append((i[0], i[1]))
        edgesX.append(i[0])
        edgesY.append(i[1])

print(edges)
fig, ax = plt.subplots()
heatmap  = ax.imshow(inverted, cmap="hot", interpolation="nearest")
fig.colorbar(heatmap)
ax.scatter(edgesY, edgesX)

In [ ]:
from scipy.ndimage import convolve

def numerical_inverse(n):
    return int(n != 1)

data = np.array([[1,1,1,1],[1,0,0,1],[1,0,0,1],[1,1,1,1]])

applyall = np.vectorize(numerical_inverse)
inverted = applyall(data)
print(inverted)

padded_array = np.pad(inverted, pad_width=1, mode='constant', constant_values=0)
kernel = [[1,1,1],[1,0,1],[1,1,1]]



print(convolve(padded_array, kernel)[1:-1, 1:-1])

In [ ]:
n = [1,2,3,4]
m = [1,2,3,4]

In [ ]:
simulation.simulate(step = 10)
print(simulation.isAtDensityEquilibirum(1.9))

densityPlot = copy.copy(simulation.density)
densityPlot[walls.invertedBoundary] -= np.min(densityPlot[walls.invertedBoundary])

fig, ax = plt.subplots()
heatmap = ax.imshow(densityPlot, cmap = "rainbow", interpolation = "nearest")
quivermap = ax.quiver(simulation.xIndex, simulation.yIndex, simulation.momentumX, -simulation.momentumY)
fig.colorbar(heatmap)
# print(simulation.fluid)
plt.show()